In [41]:
import pandas as pd
import os

# Read data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/AI%20Fundamentals%20(MMAI%205000S)/AI-Human%20Pew%20Data.csv")

# Define the SMALG columns & Demographic Columns
smalg_columns = ['SMALG2_W99', 'SMALG4_a_W99', 'SMALG4_b_W99', 'SMALG4_c_W99', 
                 'SMALG4_d_W99', 'SMALG7_W99', 'SMALG11_W99', 'SMALG12_W99']

demographic_columns = ['F_INTFREQ', 'F_RELCOM3CAT', 'F_METRO', 'F_CREGION', 'F_CDIVISION', 'F_AGECAT',
                     'F_GENDER', 'F_EDUCCAT', 'F_EDUCCAT2', 'F_HISP', 
                     'F_YEARSINUS', 'F_RACECMB', 'F_RACETHNMOD', 'F_CITIZEN', 'F_BIRTHPLACE',
                     'F_MARITAL', 'F_RELIG', 'F_BORN', 'F_RELIGCAT1', 'F_ATTEND', 'F_RELIMP',
                     'F_PRAY', 'F_PARTY_FINAL', 'F_PARTYSUM_FINAL',
                     'F_PARTYSUMIDEO_FINAL', 'F_INC_SDT1', 'F_REG', 'F_IDEO', 'F_VOLSUM', 'F_INC_TIER2']

# Select the specified columns
selected_columns = smalg_columns + demographic_columns

df_selected = df[selected_columns]

# Rename the new columns to start with "DEM_"
rename_dict = {col: f"DEM_{col}" for col in selected_columns if col.startswith('F_')}
df_selected = df_selected.rename(columns=rename_dict)

# Filter out null values from SMALG columns
df_selected = df_selected.dropna(subset=smalg_columns)

# Filter out "Refused" responses from SMALG columns
for col in smalg_columns:
    df_selected = df_selected[df_selected[col] != 'Refused']

# Replace nulls and "Refused" with specified values for DEM columns
df_selected['DEM_F_INTFREQ'] = df_selected['DEM_F_INTFREQ'].replace(['Refused', None, ''], 'Several times a day')
df_selected['DEM_F_RELCOM3CAT'] = df_selected['DEM_F_RELCOM3CAT'].replace('DK/Ref', 'Medium')
df_selected['DEM_F_AGECAT'] = df_selected['DEM_F_AGECAT'].replace('Refused', '30-49')
df_selected['DEM_F_GENDER'] = df_selected['DEM_F_GENDER'].replace('Refused', 'In some other way')

# Replace everything not "Republican" or "Democrat" with "Other" in DEM_F_PARTY_FINAL
df_selected['DEM_F_PARTY_FINAL'] = df_selected['DEM_F_PARTY_FINAL'].apply(lambda x: x if x in ['Republican', 'Democrat'] else 'Other')

# Mapping for specified columns
mapping = {
    'SMALG2_W99': {'Good idea for society': 1, 'Not sure': 0.5, 'Bad idea for society': 0},
    'SMALG4_a_W99': {'Definitely happening': 0, 'Probably happening': 0.33, 'Probably NOT happening': 0.67, 'Definitely NOT happening': 1},
    'SMALG4_b_W99': {'Definitely happening': 0, 'Probably happening': 0.33, 'Probably NOT happening': 0.67, 'Definitely NOT happening': 1},
    'SMALG4_c_W99': {'Definitely happening': 1, 'Probably happening': 0.67, 'Probably NOT happening': 0.33, 'Definitely NOT happening': 0},
    'SMALG4_d_W99': {'Definitely happening': 1, 'Probably happening': 0.67, 'Probably NOT happening': 0.33, 'Definitely NOT happening': 0},
    'SMALG7_W99': {'A great deal of confidence': 1, 'A fair amount of confidence': 0.67, 'Not too much confidence': 0.33, 'No confidence at all': 0},
    'SMALG11_W99': {'Mostly made by computer programs': 1, 'Not sure': 0.5, 'A mix of both people and computer programs': 0.5, 'Mostly made by people': 0},
    'SMALG12_W99': {'A better job than humans': 1, 'About the same job as humans': 0.5, 'Not sure': 0.5, 'A worse job than humans': 0}
}

# Apply the mappings
for col, map_dict in mapping.items():
    df_selected[col] = df_selected[col].map(map_dict)

# Define the explicit path to the home directory
home_path = os.path.expanduser("~")

# Define the output file name
output_file = "filtered_pew_data.csv"

# Create the full path for the output file
output_path = os.path.join(home_path, output_file)

# Export the DataFrame to CSV
df_selected.to_csv(output_path, index=False)

print(f"CSV file has been exported to: {output_path}")

CSV file has been exported to: C:\Users\Numan\filtered_pew_data.csv
